<a href="https://colab.research.google.com/github/sigvehaug/ISDAwPython_day3.1/blob/main/ISDAwPython_3_1_NB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Statistics with Python, S. Haug, University of Bern. 

# Parameter estimation / regression with Python

**Average expected study time :** 45 min (depending on your background)

**Learning outcomes :**

- Refresh what is meant with parameter estimation and regression
- Perform linear regression with Python by example
- Fitting to built-in functions
- Fitting to own defined functions
- Know what non-parametric regression is 

**Main python module used**
- the Scipy.stat module https://docs.scipy.org/doc/scipy/reference/stats.html


# 3.0 Regression - Situation

We have data and want to extract model paramters from that data. An example would be to estimate the mean and the standard deviation, assuming a normal distribution. Another one would be to fit a straight line. For historical reasons this kind of analysis is often called regression. Some scientists just say fitting (model parameters to the data).

We distinguish between parametric and non-parametric models. A line and the normal distribution are both parametric.

## 3.1 About linear Regression

Linear regression means fitting linear parameters to a set of data points (x,y). x and y may be vectors. You may consider this as the simplest case of Machine Learning. Example, a line is described by

$$y = ax + b$$

Thus two parameters a (slope) and b (intersection with y axis) can be fitted to (x,y). This is called linear regression as the parameters are linear (no higher powers).

There are different fitting methods, mostly least squares or maximum likelihood are used.


## 3.2 Linear regression with scipy.stats

Import the Python libraries we need.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

Read the data from file and do a linear regression for a line in the plength-pwidth space of the setosa sample. We use https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html, using least squares. 

In [1]:
url = 'https://www.openml.org/data/get_csv/61/dataset_61_iris.arff'


The number of digits is ridiculous. Let's print it better.

Let's look at the scatter plot to see if this makes sense.

By eye it is hard to say how good this fit is. Try the same regression  with versicolor. The result may be a bit clearer.

We now have a model, a straight line, whose shape we have chosen, but whose parameters (slope and intersection) have been estimated/fitted from data with the least squares method. It tells us that petal width of a leaf is petal length x slope ( f(petal length) = a x slope). So we can do interpolation and extrapolation, i.e. get the petal width at any petal length.




## 3.3 Fitting data to other built-in p.d.f.

The scipy.stats comes with many built-in functions. For example the exponential distributions with scale $\beta$ and location $\mu$

$$f(x)=\frac{1}{\beta} e^{-(x-\mu)/\beta}     ,  x \ge \mu;\beta>0$$

In [2]:
# Let us fit data to an exponential distribution


This fit method is poor in the sense that it doesn't return uncertainties on the fitted values. This we normally want to know. The curve_fit method below also returns the uncertainties.

## 3.4 Fitting your own defined function

If a line is not streight it is curved. There are many mathematical functions whose parameters we can try to fit to experimental data points. Some examples: Polynominals (first order is linear regression, second order is a parabola etc), exponential functions, normal function, sindoial wave function etc. You need to choose an approriate shape/function to obtain a good result. 

With the Scipy.stat module we can look for preprogrammed functions (in principle you can program your own function whose parameters you want to fit too): https://docs.scipy.org/doc/scipy/reference/stats.html. 

The scipy.optimize module provides a more general non-linear least squares fit. Look at and play with this example. It is complex and you will probably need some time testing, googling etc.

In [3]:
# First let us generate some synthetic data to play with
from scipy.optimize import curve_fit
# We define our own model


In [4]:
# Now use curve_fit to fit the model parameters to the data


## 3.5 Regression with statsmodels

The regression methods in scipy.stats don't give very rich output. In particular, one often would like two know more about the uncertainties on the fitted parameters. The **statsmodels** library is in this sense more powerful. Let us look at one example.

statsmodels documentation: https://www.statsmodels.org/

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

np.random.seed(9876789)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
# pwidths and plenghts we extracted from the Iris set above


## 3.6 Comment on non-parametric regression

So far we have used functions (models) with some predefined shape/form. The parameters we fitted to data. If we have no clue about the form, we may try to fit with non-parametric methods. However, these require more data as also the shape needs to guessed or fitted from the data. So normally a non-parametric method gives poorer results. 

There are several ways to do this in Python. You make look at this if you are interested:

https://pythonhosted.org/PyQt-Fit/NonParam_tut.html